Please first run:

%make install

in a terminal in this directory, and configure your runtime (VSCode, datahub, etc) to use the resultant virtual Python environment, or else uncomment the lines in the following cell

In [15]:
# pip install -r requirements.txt

In [16]:
# API key for Gov; substitute your own!
apikey = "rfaOy1m0PNZUGET08nKJvgbOrtpJaP86d8yt6RYh"

In [17]:
import sys
sys.path.append('./src')
from team_harriette_chick import getPrices
import pandas as pd

In [18]:
# from eep153_tools.sheets import read_sheets
import pandas as pd
import pickle

DRIs = None
with open('data/diet.pickle', 'rb') as fp:
    DRIs = pickle.load(fp)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

diet_max = DRIs['diet_maximums'].set_index('Nutrition')


In [ ]:
import pandas as pd
import fooddatacentral as fdc
import warnings
import pandas as pd
import numpy as np
from  scipy.optimize import linprog as lp
import numpy as np
import warnings

bulk_prices_w_lbs = pd.read_csv("data/bulk_prices.tsv", sep='\t')

apikey = "rfaOy1m0PNZUGET08nKJvgbOrtpJaP86d8yt6RYh"

D = {}
count = 0
food_names = bulk_prices_w_lbs.Material_Descr.tolist()
for food_name in food_names:
    try:
        fdc_id = bulk_prices_w_lbs.loc[bulk_prices_w_lbs.Material_Descr==food_name,:].FDC_ID
        int_fdcid = int(fdc_id[count])
        count+=1
        D[food_name] = fdc.nutrients(apikey,int_fdcid).Quantity
        print(f'count: {count} {food_name}')
    except AttributeError:
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food_name,int_fdcid))
    except ValueError:
        warnings.warn(f"No code for food {food_name}")
    except Exception as ex:
        warnings.warn(f"something seriously wrong for {food_name}  {ex}")

FoodNutrients = pd.DataFrame(D,dtype=float)

FoodNutrients = pd.DataFrame(D,dtype=float)

# all items are 1 pound
bulk_prices_w_lbs['FDC Quantity'] = bulk_prices_w_lbs[['Quantity','BaseUoM']].T.apply(lambda x : fdc.units(x['Quantity'],x['BaseUoM']))

bulk_prices_w_lbs['FDC Price'] = bulk_prices_w_lbs['Average_Price']/bulk_prices_w_lbs['FDC Quantity']

bulk_prices_w_lbs.dropna(how='any')

# To use minimum price observed
Prices = bulk_prices_w_lbs.groupby('Material_Descr',sort=False)['FDC Price'].min()


In [25]:
# our target group
# group = 'F 9-13'
group = 'M 14-18'
# tolerance
tol = 1e-6

result = getPrices.solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 14-18 is $1.42 per day.


Diet (in 100s of grams or milliliters):
EGGS WHOLE LIQ BULK -TANK                   1.361669
APPLE SLICES CAN-6/10                       0.030973
BEANS PINTO DRY PKG-12/2 LB                 1.215206
FLOUR WHOLE WHEAT BAG-50 LB                 1.038278
OIL VEGETABLE BTL-6/1 GAL                   0.581914
FLOUR BREAD-BULK                            1.046810
FLOUR WHITE WHOLE WHEAT BLEND BAG-25 LB     0.614372
POTATO FOR PROCESS INTO DEHY PRD-BULK       2.223404
CHEESE NAT AMER FBD BARREL-500 LB(40800)    0.687821
SPINACH CHOPPED FRZ IQF CTN-20 LB (1902)    1.609298
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          3099.999998          2200.0
Protein                          100.077669            52.0
Fiber, total dietary              30.800000            30.8
Folate, DFE    

### Look up nutritional information for foods



## Effects of Price Changes on Subsistence Diet Cost



As prices change, we should expect the minimum cost diet to also
  change.  The code below creates a graph which changes prices away
  from the \`base&rsquo; case one food at a time, and plots changes in total
  diet cost.



In [27]:
import cufflinks as cf
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

cost0 = getPrices.solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = getPrices.solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        cost[Prices.index[i]] = np.log(result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T
Price_response.iplot(xTitle='change in log price',yTitle='change in log cost')

/var/folders/9r/yl209mt550g4h4t5v79dcyw80000gr/T/ipykernel_15307/2131278476.py:13: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

/var/folders/9r/yl209mt550g4h4t5v79dcyw80000gr/T/ipykernel_15307/2131278476.py:13: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

/var/folders/9r/yl209mt550g4h4t5v79dcyw80000gr/T/ipykernel_15307/2131278476.py:13: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`

/var/folders/9r/yl209mt550g4h4t5v7

## Effects of Price Changes on Subsistence Diet Composition



The code below creates a graph which changes prices just for *one* food,
  and traces out the effects of this change on all the foods consumed.



In [21]:
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'EGGS WHOLE LIQ BULK -TANK'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = getPrices.solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = getPrices.solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

/Users/christophernicholas/src/EEP153/EEP153_Harriette_Chick_Project2/venv/lib/python3.12/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



## Effects of Price Changes on Subsistence Diet Nutrition



The code below creates a graph which uses the food price changes
  described above, but maps into nutrients.



In [28]:
# Matrix product maps quantities of food into quantities of nutrients
NutrientResponse = Diet_response@FoodNutrients.T

# Drop columns of missing nutrients
NutrientResponse = NutrientResponse.loc[:,NutrientResponse.count()>0]
NutrientResponse.iplot(xTitle='%s Price' % ReferenceGood,yTitle='Nutrients')

### Price Changes and Subsistence Diet Composition with Weight Constraint



Re-do our analysis of changing prices, but with a constraint that
  total weight of diet must be less that 12 hectograms (1.2 kg).



In [22]:
# import cufflinks as cf
# cf.go_offline()

ReferenceGood = 'EGGS WHOLE LIQ BULK -TANK'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = getPrices.solve_subsistence_problem(FoodNutrients,Prices,
                                  diet_min[group],diet_max[group],max_weight=12,tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = getPrices.solve_subsistence_problem(FoodNutrients,my_p,
                                       diet_min[group],diet_max[group],max_weight=12,tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

/Users/christophernicholas/src/EEP153/EEP153_Harriette_Chick_Project2/venv/lib/python3.12/site-packages/pandas/core/dtypes/cast.py:1598: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

